# Background removal with RMBG v1.4 and OpenVINO running in DeGirum PySDK

Background matting is the process of accurately estimating the foreground object in images and videos. It is a very important technique in image and video editing applications, particularly in film production for creating visual effects. In case of image segmentation, we segment the image into foreground and background by labeling the pixels. Image segmentation generates a binary image, in which a pixel either belongs to foreground or background. However, Image Matting is different from the image segmentation, wherein some pixels may belong to foreground as well as background, such pixels are called partial or mixed pixels. In order to fully separate the foreground from the background in an image, accurate estimation of the alpha values for partial or mixed pixels is necessary.


### RMBG v1.4

RMBG v1.4 is a background removal model, designed to effectively separate foreground from background in a range of categories and image types. This model has been trained on a carefully selected dataset, which includes: general stock images, e-commerce, gaming, and advertising content, making it suitable for commercial use cases powering enterprise content creation at scale. The accuracy, efficiency, and versatility currently rival leading source-available models.

## Requirements

In [ ]:
import cv2
import matplotlib.pyplot as plt
import numpy as np

import degirum as dg
import degirum_tools as dgtools

model_name = 'rmbg_background_removal--1024x1024_float_openvino_cpu_1'
zoo_url = 'https://cs.degirum.com/degirum/openvino_demos'
img_path = 'example_input.jpg'

zoo = dg.connect(dg.CLOUD, zoo_url, dgtools.get_token())
model = zoo.load_model(model_name)
img_rgb = cv2.cvtColor(cv2.imread(img_path), cv2.COLOR_BGR2RGB)

## Run inference on an image

In [ ]:
# Preprocess
h_in, w_in, c_in = img_rgb.shape

# Inference
input = cv2.resize(img_rgb, (1024,1024)).astype(np.float32) / 255
input = input.transpose((2, 0, 1))[np.newaxis, ...]
res = model(input)
res = res.results[0]['data']

# Post-process
mask = cv2.resize(np.squeeze(res), (w_in, h_in))[..., np.newaxis]
masked_image = (img_rgb * mask).astype(np.uint8)

# Show results
fig = plt.figure(figsize=(12, 12))
ax1 = fig.add_subplot(1, 2, 1)
plt.title("Input", fontsize=20)
ax1.axis("off")
ax2 = fig.add_subplot(1, 2, 2)
plt.title("Masked", fontsize=20)
ax2.axis("off")
ax1.imshow(img_rgb)
ax2.imshow(masked_image)
plt.show()